In [1]:
import matplotlib.pyplot as plt
import numpy as np
import dask.array as da
import zarr as zr
import django
from dask.distributed import Client, progress

django.setup()
from bioconverter.models import Representation

Debugging build


2020-01-23 15:21:52,153 [elements.extenders] INFO     Making DataArrays Look Beautiful
2020-01-23 15:21:52,154 [elements.extenders] INFO     Extending DataArray


In [2]:
c = Client()
c

Client Scheduler: tcp://daskscheduler:8786 Dashboard: http://daskscheduler:8787/status,Cluster Workers: 1 Cores: 4 Memory: 4.66 GB


In [3]:
it = Representation.distributed.filter(zarr__isnull=False, name__startswith="Max")

In [4]:
it.first().array

<xarray.DataArray 'data' (x: 1256, y: 1256, c: 2, t: 1)>
dask.array<zarr, shape=(1256, 1256, 2, 1), dtype=float64, chunksize=(157, 314, 1, 1)>
Coordinates:
  * c         (c) int64 0 1
    channels  (c) [('Index', '<i8'), ('Name', '<U250'), ('EmissionWavelength', '<f8'), ('ExcitationWavelength', '<f8'), ('IlluminationType', '<U250'), ('AcquisitionMode', '<U250'), ('Color', '<U250'), ('SamplesPerPixel', '<i8')] dask.array<chunksize=(2,), meta=np.ndarray>
    physt     (t) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    physx     (x) float64 dask.array<chunksize=(1256,), meta=np.ndarray>
    physy     (y) float64 dask.array<chunksize=(1256,), meta=np.ndarray>
  * t         (t) int64 0
  * x         (x) int64 0 1 2 3 4 5 6 7 ... 1249 1250 1251 1252 1253 1254 1255
  * y         (y) int64 0 1 2 3 4 5 6 7 ... 1249 1250 1251 1252 1253 1254 1255
Attributes:
    scan:        [{'DimensionOrder': 'XYCZT', 'PhysicalSizeX': 0.049345442271...
    seriesname:  APV-CA1-#1_AnkG-488_Neun594 #1

In [7]:
nana = it.first().array

In [8]:
nana.biometa.channels

,Index,Name,EmissionWavelength,ExcitationWavelength,IlluminationType,AcquisitionMode,Color,SamplesPerPixel
0,0,Ch1-T1,518.0,488.0,Epifluorescence,LaserScanningConfocalMicroscopy,16711935,1
1,1,Ch1-T2,585.0,555.0,Epifluorescence,LaserScanningConfocalMicroscopy,-16776961,1


In [6]:
nana.compute()

<xarray.DataArray 'data' (x: 1024, y: 1024, channels: 3, z: 21, time: 1)>
array([[[[[0.0000e+00],
          [0.0000e+00],
          [0.0000e+00],
          ...,
          [0.0000e+00],
          [0.0000e+00],
          [0.0000e+00]],

         [[0.0000e+00],
          [0.0000e+00],
          [0.0000e+00],
          ...,
          [0.0000e+00],
          [0.0000e+00],
          [4.5776e-05]],

         [[1.6785e-04],
          [3.3569e-04],
          [2.4414e-04],
          ...,
          [3.0518e-05],
          [1.2207e-04],
          [2.5940e-04]]],


        [[[0.0000e+00],
          [3.0518e-05],
          [7.6294e-04],
          ...,
          [1.3733e-04],
          [4.7302e-04],
          [0.0000e+00]],

         [[0.0000e+00],
          [1.4801e-03],
          [1.2817e-03],
          ...,
          [2.1667e-03],
          [4.4250e-04],
          [1.2817e-03]],

         [[0.0000e+00],
          [0.0000e+00],
          [3.9673e-04],
          ...,
          [4.5776e-04],
          [3.2043e-04],
          [7.9346e-04]]],


        [[[0.0000e+00],
          [3.6621e-04],
          [2.0447e-03],
          ...,
          [4.4250e-04],
          [8.3923e-04],
          [4.8828e-04]],

         [[2.5940e-04],
          [1.2512e-03],
          [1.6632e-03],
          ...,
          [1.7395e-03],
          [2.1515e-03],
          [1.6632e-03]],

         [[0.0000e+00],
          [3.0518e-05],
          [3.5095e-04],
          ...,
          [3.0518e-05],
          [3.2043e-04],
          [7.6294e-05]]],


        ...,


        [[[1.5259e-05],
          [5.9509e-04],
          [3.6621e-03],
          ...,
          [2.3193e-03],
          [3.0670e-03],
          [3.8757e-03]],

         [[6.7139e-04],
          [1.2970e-03],
          [5.1270e-03],
          ...,
          [7.2174e-03],
          [6.3934e-03],
          [8.4534e-03]],

         [[1.9836e-04],
          [9.1553e-05],
          [1.2817e-03],
          ...,
          [2.1515e-03],
          [1.7395e-03],
          [2.9297e-03]]],


        [[[1.6785e-04],
          [8.2397e-04],
          [1.0376e-03],
          ...,
          [3.3722e-03],
          [4.5776e-03],
          [4.5624e-03]],

         [[1.4496e-03],
          [1.5564e-03],
          [4.4556e-03],
          ...,
          [6.4697e-03],
          [7.5531e-03],
          [5.5695e-03]],

         [[2.1362e-04],
          [3.0518e-04],
          [3.9673e-04],
          ...,
          [1.6174e-03],
          [2.7618e-03],
          [1.3885e-03]]],


        [[[2.4414e-04],
          [2.1362e-04],
          [1.5564e-03],
          ...,
          [3.2959e-03],
          [4.3488e-03],
          [3.3417e-03]],

         [[9.0027e-04],
          [2.2125e-03],
          [2.3193e-03],
          ...,
          [7.4158e-03],
          [9.1705e-03],
          [6.8512e-03]],

         [[1.5259e-05],
          [0.0000e+00],
          [1.1444e-03],
          ...,
          [1.5564e-03],
          [2.5482e-03],
          [2.2736e-03]]]],



       [[[[0.0000e+00],
          [0.0000e+00],
          [0.0000e+00],
          ...,
          [0.0000e+00],
          [0.0000e+00],
          [0.0000e+00]],

         [[0.0000e+00],
          [0.0000e+00],
          [0.0000e+00],
          ...,
          [0.0000e+00],
          [0.0000e+00],
          [0.0000e+00]],

         [[1.3733e-04],
          [3.5095e-04],
          [3.3569e-04],
          ...,
          [3.2043e-04],
          [1.5259e-04],
          [4.4250e-04]]],


        [[[6.8665e-04],
          [1.5259e-04],
          [9.7656e-04],
          ...,
          [5.3406e-04],
          [1.2512e-03],
          [0.0000e+00]],

         [[1.0681e-04],
          [1.9836e-04],
          [2.3193e-03],
          ...,
          [1.3733e-03],
          [7.4768e-04],
          [0.0000e+00]],

         [[0.0000e+00],
          [1.5259e-04],
          [7.6294e-05],
          ...,
          [2.8992e-04],
          [0.0000e+00],
          [3.3569e-04]]],


        [[[3.3569e-04],
     

In [18]:
nana.max(axis=3)

<xarray.DataArray 'data' (x: 1024, y: 1024, channels: 3, time: 1)>
dask.array<nanmax-aggregate, shape=(1024, 1024, 3, 1), dtype=float16, chunksize=(256, 256, 1, 1), chunktype=numpy.ndarray>
Coordinates:
  * channels  (channels) <U1 'R' 'G' 'B'
  * time      (time) int64 0
  * x         (x) float64 0.0 0.65 1.3 1.95 2.6 ... 663.0 663.6 664.3 665.0
  * y         (y) float64 0.0 0.65 1.3 1.95 2.6 ... 663.0 663.6 664.3 665.0

In [5]:
arrays = it.asArrays().map(lambda x: x.sel(time=0)).filter(lambda x: len(x.shape) == 4).filter(lambda item: item.shape[3] == 21)

In [6]:
arrays.compute()

[<xarray.DataArray 'data' (x: 1024, y: 1024, channels: 3, z: 21)>
 dask.array<getitem, shape=(1024, 1024, 3, 21), dtype=float16, chunksize=(256, 256, 1, 11), chunktype=numpy.ndarray>
 Coordinates:
   * channels  (channels) <U1 'R' 'G' 'B'
     time      int64 0
   * x         (x) float64 0.0 0.65 1.3 1.95 2.6 ... 663.0 663.6 664.3 665.0
   * y         (y) float64 0.0 0.65 1.3 1.95 2.6 ... 663.0 663.6 664.3 665.0
   * z         (z) float64 0.0 0.65 1.3 1.95 2.6 ... 10.4 11.05 11.7 12.35 13.0
 Attributes:
     units:    µm,
 <xarray.DataArray 'data' (x: 1024, y: 1024, channels: 3, z: 21)>
 dask.array<getitem, shape=(1024, 1024, 3, 21), dtype=float16, chunksize=(256, 256, 1, 11), chunktype=numpy.ndarray>
 Coordinates:
   * channels  (channels) <U1 'R' 'G' 'B'
     time      int64 0
   * x         (x) float64 0.0 0.65 1.3 1.95 2.6 ... 663.0 663.6 664.3 665.0
   * y         (y) float64 0.0 0.65 1.3 1.95 2.6 ... 663.0 663.6 664.3 665.0
   * z         (z) float64 0.0 0.65 1.3 1.95 2.6 ... 10.

In [7]:
import dask.array as da
s = da.stack(arrays, axis=0)

In [8]:
s

dask.array<stack, shape=(5, 1024, 1024, 3, 21), dtype=float16, chunksize=(1, 256, 256, 1, 11), chunktype=numpy.ndarray>

In [13]:
new = s.persist()

In [16]:
new = new * 2

In [17]:
new.persist()

dask.array<mul, shape=(5, 1024, 1024, 3, 21), dtype=float16, chunksize=(1, 256, 256, 1, 11), chunktype=numpy.ndarray>